# Quantum associative memory

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

[`QrackNeuron`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_neuron.py) exposes the `QNeuron` class of the C++ Qrack library. With this class, the synaptic cleft is modeled as a single qubit, which might be a subsystem of a larger pure state. "Uniformly controlled" or "single-qubit-target multiplexer" gates condition a single output qubit on the general quantum state of an abitrarily large number of input qubits.

In [2]:
import math
from pyqrack import QrackSimulator, QrackNeuron, NeuronActivationFn

qsim_ex = QrackSimulator(2)
qneuron = QrackNeuron(qsim_ex, [0], 1)
qneuron.set_angles([0.0, math.pi])

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir


This neuron, with these synaptic parameters, is equivalent to a CNOT gate.

In [3]:
for perm in range(2):

    # Set input
    qsim_ex.reset_all()
    if perm & 1:
        qsim_ex.x(0)

    # Feed-forward
    qneuron.predict(r=False)

    # Measure output
    comp = qsim_ex.m(1)

    print("Input: ", perm, ", Output: ", comp)

Input:  0 , Output:  0
Input:  1 , Output:  1


After the above, the following code always produces a Bell pair that would collapse into the same output value as input, for the Hadamard initialization of the input qubit, to activate both |0> and |1> input synaptic parameters at once.

In [4]:
# Feed-forward
qsim_ex.reset_all()
qsim_ex.h(0)
qneuron.predict(r=False)
print("Input: ", qsim_ex.m(0), ", Output: ", qsim_ex.m(1))

Input:  0 , Output:  0


An `eta` value of 1/2 will fully train the default ("ignorant") state of a synaptic parameter between the combination of inputs and output. We can train a single `QrackNeuron` instance to recognize when its inputs, as an integer, are a power of two.

In [5]:
eta = 1 / 2
input_count = 4
input_power = 1 << input_count
input_log = 2

def set_permutation(qsim, perm, len):
    qsim.reset_all()
    for i in range(len):
        if (perm >> i) & 1:
            qsim.x(i)

qsim = QrackSimulator(input_count + 1)

input_indices = list(range(input_count))

q_perceptron = QrackNeuron(qsim, input_indices, input_count, NeuronActivationFn.Generalized_Logistic, alpha=10.0)

# Train the network to recognize powers of 2
print("Learning (to recognize powers of 2)...")
for perm in range(input_power):
    print("Epoch ", (perm + 1), " out of ", input_power)

    set_permutation(qsim, perm, input_count + 1)

    isPowerOf2 = ((perm != 0) and ((perm & (perm - 1)) == 0))
    q_perceptron.learn_permutation(eta, isPowerOf2)

print()
print("Should be close to 1 for powers of two, and close to 0 for all else...")
for perm in range(input_power):

    set_permutation(qsim, perm, input_count + 1)

    print("Permutation: ", perm, ", Probability: ", q_perceptron.predict())

# Now, we prepare a superposition of all available powers of 2, to predict.
powersOf2 = [(1 << i) for i in range(input_count)]

qsim2 = QrackSimulator(input_log)

qsim.compose(qsim2, list(range(input_count + 1, input_count + 1 + input_log)))
set_permutation(qsim, 1 << (input_count + 1), input_count + 1 + input_log)
for i in range(input_log):
    qsim.h(input_count + 1 + i)
qsim.lda(list(range(input_count + 1, input_count + 1 + input_log)), list(range(0, input_count)), powersOf2)
for i in range(input_log):
    qsim.h(input_count + 1 + i)
qsim.dispose(list(range(input_count + 1, input_count + 1 + input_log)))

print()
print("(Superposition of all powers of 2) Probability: ", q_perceptron.predict())

Learning (to recognize powers of 2)...
Epoch  1  out of  16
Epoch  2  out of  16
Epoch  3  out of  16
Epoch  4  out of  16
Epoch  5  out of  16
Epoch  6  out of  16
Epoch  7  out of  16
Epoch  8  out of  16
Epoch  9  out of  16
Epoch  10  out of  16
Epoch  11  out of  16
Epoch  12  out of  16
Epoch  13  out of  16
Epoch  14  out of  16
Epoch  15  out of  16
Epoch  16  out of  16

Should be close to 1 for powers of two, and close to 0 for all else...
Permutation:  0 , Probability:  0.0
Permutation:  1 , Probability:  1.0
Permutation:  2 , Probability:  1.0
Permutation:  3 , Probability:  0.0
Permutation:  4 , Probability:  1.0
Permutation:  5 , Probability:  0.0
Permutation:  6 , Probability:  0.0
Permutation:  7 , Probability:  0.0
Permutation:  8 , Probability:  1.0
Permutation:  9 , Probability:  0.0
Permutation:  10 , Probability:  0.0
Permutation:  11 , Probability:  0.0
Permutation:  12 , Probability:  0.0
Permutation:  13 , Probability:  0.0
Permutation:  14 , Probability:  0.0
P